<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Carga

En este laboratorio, tu objetivo será diseñar la estructura de una base de datos relacional, crear las tablas necesarias y cargar en ellas los datos limpios y preparados que obtuviste en el laboratorio anterior. Trabajarás con datos relacionados con la demanda, generación eléctrica, y variables socioeconómicas, almacenándolos de manera eficiente para facilitar el análisis y las consultas posteriores.


- **Diseño de la Base de Datos:** Define una estructura de base de datos relacional que sea adecuada para almacenar los datos de demanda eléctrica, generación eléctrica, población y PIB por provincia.

- **Creación de la Base de Datos:** Escribe los scripts SQL necesarios para crear las tablas definidas en la estructura de la base de datos. Asegúrate de definir correctamente los tipos de datos y las restricciones (e.g., `NOT NULL`, `UNIQUE`).

- **Carga de Datos en la Base de Datos:** Utiliza Python para cargar los datos limpios y preparados en las tablas de la base de datos. Asegúrate de que los datos se insertan correctamente, manejando posibles errores o inconsistencias en los datos.


In [ ]:
import pandas as pd
import numpy as np
import psycopg2



ImportError: cannot import name 'InFailedSqlTransaction' from 'psycopg2' (c:\Users\Victor\anaconda3\envs\myenv\lib\site-packages\psycopg2\__init__.py)

# Cargamos los datos    

In [20]:
df_dda_electrica = pd.read_csv("../../datos_extraccion/dda_electrica_preparado.csv", index_col=0)
df_gen_electrica = pd.read_csv("../../datos_extraccion/gen_electrica_preparado.csv",index_col=0)
df_datos_demograficos = pd.read_csv("../../datos_extraccion/datos_demograficos_preparado.csv",index_col=0)
df_datos_eco = pd.read_csv("../../datos_extraccion/datos_economicos_preparado.csv",index_col=0)

In [14]:
df_dda_electrica.columns

Index(['index', 'ccaa', 'fecha_dda', 'energía_dda'], dtype='object')

# Conexion con bdd

In [49]:
#Nos conectamos a postgres
try:
    conn = psycopg2.connect(
        database = "Lab5",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432"

)
except OperationalError:
    print("la contraseña es errónea")

In [16]:

# Open a cursor to perform database operations
cur = conn.cursor()

# Create tables
create_tables = """
CREATE TABLE ccaa (
    id SERIAL PRIMARY KEY,
    name TEXT UNIQUE
);

CREATE TABLE dda_electrica (
    id SERIAL PRIMARY KEY,
    index INTEGER,
    fecha_dda DATE,
    energía_dda NUMERIC,
    ccaa_id INTEGER REFERENCES ccaa(id)
);

CREATE TABLE gen_electrica (
    id SERIAL PRIMARY KEY,
    energia_generada NUMERIC,
    percentage NUMERIC,
    fecha_generacion DATE,
    tipo_energia TEXT,
    ccaa_id INTEGER REFERENCES ccaa(id)
);

CREATE TABLE datos_demograficos (
    id SERIAL PRIMARY KEY,
    edad_grupos TEXT,
    espanoles_extranjeros TEXT,
    sexo TEXT,
    año INTEGER,
    total INTEGER,
    codigo TEXT,
    provincia TEXT,
    ccaa_id INTEGER REFERENCES ccaa(id)
);

CREATE TABLE datos_eco (
    id SERIAL PRIMARY KEY,
    año INTEGER,
    total INTEGER,
    codigo TEXT,
    provincia TEXT,
    ccaa_id INTEGER REFERENCES ccaa(id)
);
"""

# Execute the SQL command
cur.execute(create_tables)
conn.commit()
conn.close()

DuplicateTable: la relación «ccaa» ya existe


Como ha cogido mal los CSVs, volvemos a aplicar cambios, y ponemos algunas columnas en minúscula para facilitar el insert

In [ ]:
df_datos_demograficos.rename(columns = ({"Edad (3 grupos de edad)": "edad","Españoles/Extranjeros":"nacionalidad","Sexo":"sexo","Año":"anio","Total":"total"}), inplace=True)
df_datos_demograficos.columns

Index(['edad', 'nacionalidad', 'sexo', 'anio', 'total', 'codigo', 'provincia',
       'ccaa'],
      dtype='object')

In [29]:
df_datos_demograficos["total"] = df_datos_demograficos["total"].astype(str).str.replace(".","").astype(int)

In [54]:
df_datos_eco.rename(columns = ({"Total": "total"}), inplace = True)

In [55]:
#Nos conectamos a postgres
try:
    conn = psycopg2.connect(
        database = "Lab5",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432"

)
except OperationalError:
    print("la contraseña es errónea")

In [56]:
try:
    
    # Open a cursor to perform database operations
    cur = conn.cursor()

    # Insert distinct 'ccaa' values into the 'ccaa' table
    ccaa_values = ['C_LM', 'Valencia', 'Andalucia', 'Pais_Vasco', 'Asturias',
        'C_leon', 'Extremadura', 'Cataluña', 'Cantabria', 'Galicia',
        'Aragon', 'Madrid', 'Murcia', 'Navarra', 'La_rioja']

    for ccaa in ccaa_values:
        cur.execute("INSERT INTO ccaa (name) VALUES (%s) ON CONFLICT (name) DO NOTHING", (ccaa,))



    for row in df_dda_electrica.itertuples(index=False):
        cur.execute("""
            INSERT INTO dda_electrica (index, fecha_dda, energía_dda, ccaa_id)
            VALUES (%s, %s, %s, (SELECT id FROM ccaa WHERE name = %s))
        """, (row.index, row.fecha_dda, row.energía_dda, row.ccaa))

    for row in df_gen_electrica.itertuples(index=False):
        cur.execute("""
            INSERT INTO gen_electrica (energia_generada, percentage, fecha_generacion, tipo_energia, ccaa_id)
            VALUES (%s, %s, %s, %s, (SELECT id FROM ccaa WHERE name = %s))
        """, (row.energia_generada, row.percentage, row.fecha_generacion, row.tipo_energia, row.ccaa))

    for row in df_datos_demograficos.itertuples(index=False):
        cur.execute("""
            INSERT INTO datos_demograficos (edad_grupos, espanoles_extranjeros, sexo, año, total, codigo, provincia, ccaa_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, (SELECT id FROM ccaa WHERE name = %s))
        """, (row.edad, row.nacionalidad, row.sexo, row.anio, row.total, row.codigo, row.provincia, row.ccaa))

    # Iterar sobre las filas con itertuples para 'datos_eco'
    for row in df_datos_eco.itertuples(index=False):
        cur.execute("""
            INSERT INTO datos_eco (año, total, codigo, provincia, ccaa_id)
            VALUES (%s, %s, %s, %s, (SELECT id FROM ccaa WHERE name = %s))
        """, (row.año, row.total, row.codigo, row.provincia, row.ccaa))

    # Commit the changes
    conn.commit()

    # Close communication with the database
    cur.close()
    conn.close()
except psycopg2.Error as e:
    print(e)